In [79]:
#Importamos la librería base NLTK
import nltk

#Importamos el componente de NLTK para tokenizar
from nltk.tokenize import word_tokenize

#Importamos el corpus CESS en Español
from nltk.corpus import cess_esp

#Taggers HMM
from nltk.tag.hmm import HiddenMarkovModelTagger

#RegEx Parser
from nltk.chunk.regexp import *


#Esto nos permitirá crear los conjuntos de test y train
from sklearn.model_selection import train_test_split

#Importamos por último pands
import pandas as pd

#libreria json para convertir texto a json
import json

In [350]:
#Se afina el corpus a partir del original, para poder contemplar algunos escenario, incluidas las frases que se encuentran como test en el enunciado del caso practico.
corpus=[    
    [('Quiero', 'vmip1s0'), ('2', 'z'), ('billetes', 'ncmp000'), ('de', 'SPCMS'), ('Madrid', 'ncfp000'), ('a', 'sps00'), ('Frankfurt', 'np000g0'), ('en', 'sps00'), ('Septiembre', 'w')], 
    [('Necesito', 'vmip1s0'), ('comprar', 'vmip1s0'), ('un', 'di0ms0'), ('billete', 'ncms000'), ('a', 'sps00'), ('Madrid', 'np000g0'), ('el', 'tdms0'), ('5', 'w'), ('de', 'sps00'), ('Agosto', 'w')], 
    [('Comprar', 'vmn0000'), ('billete', 'ncms000'), ('Barcelona', 'ncfp000'), ('a', 'sps00'), ('Roma', 'np000g0'), ('para', 'sps00'), ('el', 'tdms0'), ('25', 'w'), ('de', 'sps00'), ('agosto', 'w'), ('con', 'sps00'), ('Iberia', 'npcso00')], 
    [('Billete', 'ncms000'), ('barato', 'aq0ms0'), ('AirEuropa', 'npcso00'), ('de', 'SPCMS'), ('Madrid', 'ncfp000'), ('a', 'sps00'), ('Sevilla', 'np000g0')],
    [('Quiero', 'vmip1s0'), ('ir', 'vmip1s0'), ('a', 'sps00'), ('Roma', 'np000g0'), ('con', 'sps00'), ('Lufthansa', 'npcso00')],    
    [('Billete', 'ncms000'), ('de', 'SPCMS'), ('Berlin', 'ncfp000'), ('a', 'sps00'), ('Madrid', 'np000g0'), ('con', 'sps00'), ('Lufthansa', 'npcso00')],
    [('Comprar', 'vmn0000'), ('tres', 'di0ms0'), ('billetes', 'ncmp000'), ('para', 'sps00'), ('el', 'tdms0'), ('15', 'w'), ('de', 'sps00'), ('octubre', 'w'), ('con', 'sps00'), ('Iberia', 'npcso00'), ('de', 'sps00'), ('Madrid', 'ncfp000'), ('a', 'sps00'), ('Bilbao', 'np000g0')]    
]

In [324]:
frases=[
    'Comprar billete Barcelona a Roma para el 25 de Agosto con Iberia',
    'Quiero 2 billetes de Madrid a Frankfurt en Septiembre',
    'Necesito comprar un billete a Madrid el 5 de Agosto',
    'Billete barato AirEuropa de Madrid a Sevilla',
    #se agregan estas frases que se encuentran como prueba en el enunciado del caso practico
    'Billete de Berlin a Madrid con Lufthansa',
    'Comprar tres billetes para el 15 de octubre con Iberia de Madrid a Bilbao'
]

### Entrenamiento con el modelo HiddenMarkovModelTagger y definicion de una funcion que permite etiquetar a las palabras

In [351]:
#entrenamiento del tagger con el corpus original modificado/mejorado su etiquedato ya que son pocos datos
tagger = HiddenMarkovModelTagger.train(corpus)
def get_tagged(peticion):
    tokens = word_tokenize(peticion, language='spanish')
    return tagger.tag(tokens)

In [343]:
# Prueba de frases de ejemplo
for peticion in frases:    
    print(peticion)
    print(get_tagged(peticion))
    print("\n")

Comprar billete Barcelona a Roma para el 25 de Agosto con Iberia
[('Comprar', 'vmn0000'), ('billete', 'ncms000'), ('Barcelona', 'ncfp000'), ('a', 'sps00'), ('Roma', 'np000g0'), ('para', 'sps00'), ('el', 'tdms0'), ('25', 'w'), ('de', 'sps00'), ('Agosto', 'w'), ('con', 'sps00'), ('Iberia', 'npcso00')]


Quiero 2 billetes de Madrid a Frankfurt en Septiembre
[('Quiero', 'vmip1s0'), ('2', 'z'), ('billetes', 'ncmp000'), ('de', 'SPCMS'), ('Madrid', 'ncfp000'), ('a', 'sps00'), ('Frankfurt', 'np000g0'), ('en', 'sps00'), ('Septiembre', 'w')]


Necesito comprar un billete a Madrid el 5 de Agosto
[('Necesito', 'vmip1s0'), ('comprar', 'vmip1s0'), ('un', 'di0ms0'), ('billete', 'ncms000'), ('a', 'sps00'), ('Madrid', 'np000g0'), ('el', 'tdms0'), ('5', 'w'), ('de', 'sps00'), ('Agosto', 'w')]


Billete barato AirEuropa de Madrid a Sevilla
[('Billete', 'ncms000'), ('barato', 'aq0ms0'), ('AirEuropa', 'npcso00'), ('de', 'SPCMS'), ('Madrid', 'ncfp000'), ('a', 'sps00'), ('Sevilla', 'np000g0')]


Billete de B

# 1. Parte básica:

Generar una tabla o JSON que contenga la siguiente información de cada mensaje:

- Para el mensaje: “Comprar tres billetes para el 15 de octubre con Iberia de Madrid a Bilbao”.
- Para la frase “Billete de Berlín a Madrid con Lufthansa”.

In [344]:
#Definicion de las reglas de expresion regular y parser
reglas = r'''
    cantidad: {<z>|<mccp00>|<di0ms0|di0fs0|di0mp0|di0fp0|dn0cp0|mcmp00|mcfp00|mcfs00> || <mcmp00>* || <mcfp00> |<aq0ms0>} 
    origen: {<ncfp000>}
    destino: {<np000g0>}
    aerolinea: {<npcso00>}
    fecha:{<w>*<sps00>*<w>}
      '''
regexParser = nltk.RegexpParser(reglas)
def parsear(phrase):
    return regexParser.parse(phrase)

In [357]:
#generar objeto con datos de vuelo
def genera_reserva_vuelo(tree):
    
    item = {}
    item["origen"] = None
    item["destino"] = None
    item["fecha"] = None    
    item["cantidad"] = 1
    item["aerolinea"] = None
            
    #Lectura de cada nodo categorizado
    for nodo in tree:
        if type(nodo) == tuple:
            continue
        
        valor = None
        valor_fecha = ''
        
        #Para el nodo fecha se enlaza cada elemento
        if nodo.label() == 'fecha':
            for elemento in nodo:
                palabra, categoria = elemento
                valor_fecha = valor_fecha.strip() + ' ' + palabra
            item["fecha"] = valor_fecha
        #Para los otros nodos se obtiene su valor correspondiente
        else:
            for elemento in nodo:                
                palabra, categoria = elemento
                valor = palabra
                if nodo.label() == 'origen':
                    item["origen"] = valor
                if nodo.label() == 'destino':
                    item["destino"] = valor
                if nodo.label() == 'cantidad':
                    item["cantidad"] = valor
                if nodo.label() == 'aerolinea':
                    item["aerolinea"] = valor         
    return item

In [358]:
#Funcion que genera el json a partir de la frase enviada
def generar_json(frase):
    frase_tagged = get_tagged(frase)
    frase_parsed = parsear(frase_tagged)    
    item = genera_reserva_vuelo(frase_parsed)
    with open('output.txt','w') as file:
        file.write(json.dumps(item))
    return item

In [359]:
#Prueba de frases del caso practico
peticion1 = "Billete de Berlin a Madrid con Lufthansa"
generar_json(peticion1)

{'origen': 'Berlin',
 'destino': 'Madrid',
 'fecha': None,
 'cantidad': 1,
 'aerolinea': 'Lufthansa'}

In [360]:
#Prueba de frases del caso practico
peticion2 = "Comprar tres billetes para el 15 de octubre con Iberia de Madrid a Bilbao"
generar_json(peticion2)

{'origen': 'Madrid',
 'destino': 'Bilbao',
 'fecha': '15 de octubre',
 'cantidad': 'tres',
 'aerolinea': 'Iberia'}

# 2. Parte intermedia:
Crear un asistente virtual. 

Al ejecutar la función asistent(), el usuario debe escuchar el mensaje “Hola, bienvenido a sky2travel. ¿Cómo te puedo ayudar?”. A continuación, indicará lo que quiere; por ejemplo, “Billete de Berlín a Madrid con Iberia”; posteriormente, se lanzará el bot creado en la parte básica y se devolverá el mensaje por voz: “Perfecto. Comienzo la búsqueda de tu viaje a X desde Y para el <fecha> con <aerolínea>”.

 La conversión de text a audio y viceversa es un plus. Al menos se debe interactuar como un chat escrito.


In [367]:
print("Hola, bienvenido a sky2travel. ¿Cómo te puedo ayudar?")
#Quiero un billete de Madrid a Bilbado para el 10 de octubre con Iberia
peticion = input()
respuesta= generar_json(peticion)
print( "Perfecto. Comienzo la búsqueda de tu viaje a ", respuesta.get("destino"), " desde " , respuesta.get("origen"), " para el ", respuesta.get("fecha")," con ",respuesta.get("aerolinea") )


Hola, bienvenido a sky2travel. ¿Cómo te puedo ayudar?
Perfecto. Comienzo la búsqueda de tu viaje a  Bilbado  desde  Madrid  para el  10 de octubre  con  Iberia


##### 3. Parte avanzada:

Una vez se han podido detectar los datos básicos del pedido, Amadeus para poder tramitar o buscar los vuelos no puede trabajar con el nombre de la ciudad, necesita el código IATA del aeropuerto. 

Para obtener dicho código habrá que hacer uso de la API abierta “Air-Port-Codes”. En la sección de Recursos puede encontrar más información sobre esta API.
En la parte final de la práctica, los alumnos deberán de generar un JSON como en la parte básica, pero incluyendo los siguientes campos:

Origen: ‘Madrid’ -> String nombre ciudad
Destino: ‘Frankfurt’ -> String nombre ciudad
IATA_FROM: ‘MAD’ -> String 3 carácteres
IATA_TO: ‘FRA’ -> String 3 carácteres
Fecha: ‘15-08-2021’ -> Fecha (formato dd-mm-yyy o string)
Pax: 3 -> Int
    
### Recursos:

En esta sección se pueden encontrar recursos de ayuda o instrucciones para hacer la práctica:

Frases de ejemplo: “frases-travel.txt”.
Corpus tagueado travel para entrenamiento: “corpus-travel.txt”.
Etiquetas EAGLES: https://www.cs.upc.edu/~nlp/tools/parole-sp.html
“Air-Port-Codes” API:
 a) Para registrarse en la API, acceder a: https://www.air-port-codes.com/auth/register/

 b) La API que se deberá utilizar es: /api/v1/multi.

 c) La API solo admite nombre de ciudades en inglés (por ejemplo, en lugar de Londres, habrá que enviarle London; en el caso de Berlín, Berlin). Está permitido utilizar cualquier librería o API para traducir nombres de ciudades si fuese necesario.